In [65]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.optim import lr_scheduler

from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import VAE,AEBase
from models import DNN
import numpy as np
import pandas as pd
import models
import utils as ut
import copy



In [66]:
from scipy.stats import pearsonr

# Parameters

In [67]:
# Define parameters
epochs = 500 #200,500,1000  
#dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1
select_drug = 'Gefitinib'
na = -1

# Import data

In [68]:
data_r=pd.read_csv('data/GDSCexpression.csv',index_col=0)
label_r=pd.read_csv('data/GDSClabel.csv',index_col=0)

In [69]:
label_r=label_r.fillna(na)

In [70]:
hvg,adata = ut.highly_variable_genes(data_r)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [71]:
label_r

,Erlotinib,AICAR,Camptothecin,Vinblastine,Cisplatin,Cytarabine,Docetaxel,Methotrexate,ATRA,Gefitinib,...,CMK,Pyrimethamine,JW-7-52-1,A-443654,GW843682X,MS-275,Parthenolide,MG-132,KIN001-135,TGX221
BxPC-3,-1.000000,0.159708,0.188393,0.436127,0.154274,0.113764,0.249987,0.015324,0.015324,0.020293,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
KMOE-2,0.002108,0.339814,0.318206,0.281740,0.063808,0.140681,0.101530,0.191210,0.031229,0.007419,...,0.022662,0.025217,0.270791,0.095608,0.139914,0.171747,0.026670,0.013904,0.002116,0.000956
MFM-223,0.002148,0.334087,0.122671,0.361438,0.051983,0.186659,0.228797,0.003639,0.023043,0.003639,...,0.038997,0.006167,0.002185,0.102592,0.020338,0.010874,0.000970,0.035206,0.007051,0.000323
NUGC-3,-1.000000,0.095099,0.416792,0.422631,0.266409,0.237811,0.234952,0.011558,0.028838,0.153002,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
OC-314,-1.000000,0.012740,0.479915,0.390145,0.067738,0.125869,0.249668,0.110418,0.002214,0.020309,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KP-N-S19s,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
BC-3,0.003515,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,0.048227,0.146476,0.400087,0.205247,0.147090,0.298803,0.023796,0.017219,0.002065,0.069181
Panc 08.13,-1.000000,0.152818,0.196279,0.180653,0.090963,0.093197,0.097267,0.018269,0.010237,0.009405,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
EKVX,0.177238,0.078395,0.078938,0.072213,0.007225,0.003470,0.129184,0.095642,0.111598,0.126364,...,0.020472,0.044047,0.243635,0.079375,0.019859,0.177759,0.014843,0.114431,0.007216,0.063911


In [72]:
selected_idx = label_r.loc[:,select_drug]!=na

In [73]:
data_r.columns = adata.var_names

# Your is gene-cell, mine is cell-gene

In [74]:
data = data_r.loc[selected_idx,hvg]

In [75]:
label = label_r.loc[selected_idx,select_drug]
scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
data = scaler.fit_transform(data)


In [76]:
print(np.std(data))
print(np.mean(data))

0.9999999999999999
-9.065550491767716e-18


In [77]:
data.shape

(675, 3462)

In [78]:
label_r.shape

(789, 139)

# Split test train

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [80]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(675, 3462)
(675,)
(432, 3462) (432,)
(135, 3462) (135,)


In [81]:
print(X_train.max())
print(X_train.min())

19.375997920629374
-7.905185436786982


# AE MODEL

In [82]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [83]:
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)
X_allTensor = torch.FloatTensor(data).to(device)

Y_trainTensor = torch.FloatTensor(Y_train.values).to(device)
Y_validTensor = torch.FloatTensor(Y_valid.values).to(device)

# construct TensorDataset
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)
test_dataset = TensorDataset(X_testTensor, X_testTensor)
all_dataset = TensorDataset(X_allTensor, X_allTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=200, shuffle=True)
X_allDataLoader = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

In [84]:
dataloader = X_trainDataLoader

In [85]:
dataloaders_train = {'train':X_trainDataLoader,'val':X_validDataLoader}

In [86]:
X_trainDataLoader.dataset.tensors[0].shape[0]

432

In [87]:
{x: dataloaders_train[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}

{'train': 432, 'val': 108}

# The model

In [88]:
model = AEBase(input_dim=data.shape[1],latent_dim=256,hidden_dims=[1024,512,256])

In [89]:
print(model)

AEBase(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3462, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout(p=0.3, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout(p=0.3, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout(p=0.3, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=256, out_features=256, bias=True)
  (decoder_input): Linear(in_features=256, out_features=256, bias=True)
  (decoder): Sequent

In [90]:
#model = VAE(dim_au_in=data_r.shape[1],dim_au_out=128)
if torch.cuda.is_available():
    model.cuda()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_function = nn.SmoothL1Loss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [91]:
def train_ae_model(net,data_loaders,optimizer,loss_function,n_epochs,scheduler):
    
    dataset_sizes = {x: data_loaders[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}
    loss_train = {}
    
    best_model_wts = copy.deepcopy(net.state_dict())
    best_loss = np.inf

    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch, n_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #optimizer = scheduler(optimizer, epoch)
                net.train()  # Set model to training mode
            else:
                net.eval()  # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            # for data in data_loaders[phase]:
            for batchidx, (x, _) in enumerate(data_loaders[phase]):

                x.requires_grad_(True)
                # encode and decode 
                output = model(x)
                # compute loss
                loss = loss_function(output, x)      

                # zero the parameter (weight) gradients
                optimizer.zero_grad()

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    # update the weights
                    optimizer.step()

                # print loss statistics
                running_loss += loss.item()
            
            # Schedular
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            
            loss_train[epoch,phase] = epoch_loss 
            print('{} Loss: {:.8f}'.format(phase, epoch_loss))
            
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
    
    # Select best model wts
    torch.save(best_model_wts, 'saved/models/ae.pkl')
    net.load_state_dict(best_model_wts)           
    
    return net, loss_train

In [94]:
model,loss_report = train_ae_model(net=model,data_loaders=dataloaders_train,
                             optimizer=optimizer,loss_function=loss_function,
                            n_epochs=epochs,scheduler=exp_lr_scheduler)

Epoch 0/499
----------
train Loss: 0.00239341
val Loss: 0.00300079
Epoch 1/499
----------
train Loss: 0.00238665
val Loss: 0.00300216
Epoch 2/499
----------
train Loss: 0.00238214
val Loss: 0.00300305
Epoch 3/499
----------
train Loss: 0.00240891
val Loss: 0.00300558
Epoch 4/499
----------
train Loss: 0.00239257
val Loss: 0.00300388
Epoch 5/499
----------
train Loss: 0.00236371
val Loss: 0.00300354
Epoch 6/499
----------
train Loss: 0.00239391
val Loss: 0.00300440
Epoch 7/499
----------
train Loss: 0.00237672
val Loss: 0.00300528
Epoch 8/499
----------
train Loss: 0.00238567
val Loss: 0.00300404
Epoch 9/499
----------
train Loss: 0.00237047
val Loss: 0.00300453
Epoch 10/499
----------
train Loss: 0.00234157
val Loss: 0.00300414
Epoch 11/499
----------
train Loss: 0.00235259
val Loss: 0.00300296
Epoch 12/499
----------
train Loss: 0.00240162
val Loss: 0.00300569
Epoch 13/499
----------
train Loss: 0.00235590
val Loss: 0.00300443
Epoch 14/499
----------
train Loss: 0.00235226
val Loss: 0

train Loss: 0.00237733
val Loss: 0.00300417
Epoch 123/499
----------
train Loss: 0.00236094
val Loss: 0.00300220
Epoch 124/499
----------
train Loss: 0.00238088
val Loss: 0.00300221
Epoch 125/499
----------
train Loss: 0.00238174
val Loss: 0.00300346
Epoch 126/499
----------
train Loss: 0.00234606
val Loss: 0.00300180
Epoch 127/499
----------
train Loss: 0.00240663
val Loss: 0.00300271
Epoch 128/499
----------
train Loss: 0.00233637
val Loss: 0.00300117
Epoch 129/499
----------
train Loss: 0.00236218
val Loss: 0.00300069
Epoch 130/499
----------
train Loss: 0.00240564
val Loss: 0.00300253
Epoch 131/499
----------
train Loss: 0.00237236
val Loss: 0.00300365
Epoch 132/499
----------
train Loss: 0.00238450
val Loss: 0.00300383
Epoch 133/499
----------
train Loss: 0.00237330
val Loss: 0.00300320
Epoch 134/499
----------
train Loss: 0.00236488
val Loss: 0.00300214
Epoch 135/499
----------
train Loss: 0.00237539
val Loss: 0.00300252
Epoch 136/499
----------
train Loss: 0.00233298
val Loss: 0

val Loss: 0.00300282
Epoch 243/499
----------
train Loss: 0.00239025
val Loss: 0.00300340
Epoch 244/499
----------
train Loss: 0.00236517
val Loss: 0.00300150
Epoch 245/499
----------
train Loss: 0.00236713
val Loss: 0.00300138
Epoch 246/499
----------
train Loss: 0.00237682
val Loss: 0.00300127
Epoch 247/499
----------
train Loss: 0.00236081
val Loss: 0.00300219
Epoch 248/499
----------
train Loss: 0.00237723
val Loss: 0.00300170
Epoch 249/499
----------
train Loss: 0.00235206
val Loss: 0.00300306
Epoch 250/499
----------
train Loss: 0.00237189
val Loss: 0.00300287
Epoch 251/499
----------
train Loss: 0.00239061
val Loss: 0.00300367
Epoch 252/499
----------
train Loss: 0.00238389
val Loss: 0.00300142
Epoch 253/499
----------
train Loss: 0.00236556
val Loss: 0.00300279
Epoch 254/499
----------
train Loss: 0.00236854
val Loss: 0.00300353
Epoch 255/499
----------
train Loss: 0.00235443
val Loss: 0.00300380
Epoch 256/499
----------
train Loss: 0.00233533
val Loss: 0.00300247
Epoch 257/499

val Loss: 0.00300489
Epoch 363/499
----------
train Loss: 0.00236488
val Loss: 0.00300463
Epoch 364/499
----------
train Loss: 0.00238138
val Loss: 0.00300554
Epoch 365/499
----------
train Loss: 0.00234251
val Loss: 0.00300493
Epoch 366/499
----------
train Loss: 0.00236695
val Loss: 0.00300420
Epoch 367/499
----------
train Loss: 0.00238464
val Loss: 0.00300515
Epoch 368/499
----------
train Loss: 0.00237343
val Loss: 0.00300387
Epoch 369/499
----------
train Loss: 0.00239682
val Loss: 0.00300380
Epoch 370/499
----------
train Loss: 0.00236651
val Loss: 0.00300459
Epoch 371/499
----------
train Loss: 0.00236379
val Loss: 0.00300276
Epoch 372/499
----------
train Loss: 0.00239560
val Loss: 0.00300426
Epoch 373/499
----------
train Loss: 0.00237767
val Loss: 0.00300471
Epoch 374/499
----------
train Loss: 0.00237079
val Loss: 0.00300223
Epoch 375/499
----------
train Loss: 0.00238227
val Loss: 0.00300320
Epoch 376/499
----------
train Loss: 0.00236372
val Loss: 0.00300131
Epoch 377/499

val Loss: 0.00300125
Epoch 483/499
----------
train Loss: 0.00237614
val Loss: 0.00300182
Epoch 484/499
----------
train Loss: 0.00238978
val Loss: 0.00300207
Epoch 485/499
----------
train Loss: 0.00235152
val Loss: 0.00300139
Epoch 486/499
----------
train Loss: 0.00235882
val Loss: 0.00300141
Epoch 487/499
----------
train Loss: 0.00234249
val Loss: 0.00300275
Epoch 488/499
----------
train Loss: 0.00234595
val Loss: 0.00300086
Epoch 489/499
----------
train Loss: 0.00239838
val Loss: 0.00300164
Epoch 490/499
----------
train Loss: 0.00238936
val Loss: 0.00300218
Epoch 491/499
----------
train Loss: 0.00238117
val Loss: 0.00300310
Epoch 492/499
----------
train Loss: 0.00237789
val Loss: 0.00300425
Epoch 493/499
----------
train Loss: 0.00236517
val Loss: 0.00300451
Epoch 494/499
----------
train Loss: 0.00238074
val Loss: 0.00300431
Epoch 495/499
----------
train Loss: 0.00237997
val Loss: 0.00300374
Epoch 496/499
----------
train Loss: 0.00239246
val Loss: 0.00300293
Epoch 497/499

In [93]:
loss_report

{(0, 'train'): 0.002915243928631147,
 (0, 'val'): 0.003232359610222004,
 (1, 'train'): 0.002734291332739371,
 (1, 'val'): 0.003227505694936823,
 (2, 'train'): 0.0027137183480792576,
 (2, 'val'): 0.0032197579189583107,
 (3, 'train'): 0.002724611027925103,
 (3, 'val'): 0.0032085621798479996,
 (4, 'train'): 0.002628321310988179,
 (4, 'val'): 0.0031893167782712866,
 (5, 'train'): 0.002519234463020607,
 (5, 'val'): 0.003155301566477175,
 (6, 'train'): 0.0025244680129819447,
 (6, 'val'): 0.0031132778083836593,
 (7, 'train'): 0.002490781730523816,
 (7, 'val'): 0.0030793682844550523,
 (8, 'train'): 0.0024758236551726304,
 (8, 'val'): 0.0030545357752729345,
 (9, 'train'): 0.0024284741116894614,
 (9, 'val'): 0.00303453896884565,
 (10, 'train'): 0.0024239554035442846,
 (10, 'val'): 0.0030300573066428856,
 (11, 'train'): 0.00239370404570191,
 (11, 'val'): 0.003025652633772956,
 (12, 'train'): 0.0024084121816688115,
 (12, 'val'): 0.0030221116763574107,
 (13, 'train'): 0.0024161607854896123,
 (13, '

In [ ]:
# Train autoencoder

for epoch in range(epochs):
    # 不需要label，所以用一个占位符"_"代替
    for batchidx, (x, _) in enumerate(X_allDataLoader):
        x.requires_grad_(True)
        # encode and decode 
        output = model(x)
        # compute loss
        print(output.shape)
        loss = loss_function(output, x)      
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item()))

In [ ]:
import os
os.getcwd()

In [ ]:
torch.save(model.state_dict(), 'saved/models/ae.pkl')

In [ ]:
recon_batch = model(trainData)

In [ ]:
loss_function(recon_batch,trainData)

In [ ]:
train_embeddings = model.encode(trainData)

In [ ]:
feature = train_embeddings.cpu().detach().numpy()

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.001)
clf.fit(feature, Y_train.values)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFrg = RandomForestRegressor(random_state=0, n_estimators=100)
RFrg.fit(feature, Y_train.values)

In [ ]:
testFeature = model.encode(testData)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [ ]:
rfresult = RFrg.predict(testFeature.detach().cpu().numpy())

In [ ]:
r2_score(lasso,Y_test)

In [ ]:
r2_score(rfresult,Y_test)

In [ ]:
Y_test.values

In [ ]:
rfresult

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

In [ ]:
tsne_results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figures/tsne_vae_gdsc.png")

In [ ]:
EPOCH = 500

In [ ]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [ ]:
# Initialization DNN model

predictor = DNN(128, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
loss_func = nn.MSELoss().to(device)
#loss2-sigmoid
#loss_func = nn.BCELoss()
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [ ]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 

# Save model
torch.save(predictor.state_dict(), 'saved/models/DNN_GDSC.pkl')

In [ ]:
testpredict = predictor(testFeature)

In [ ]:
testpredict

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

In [ ]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)